# Capstone

This notebook is going to be used for Coursera Capstone project.

In [2]:
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


To complete capstone, you will be working on a case study which is to predict the severity of an accident. Say you are driving to another city for work or to visit some friends. It is rainy and windy, and on the way, you come across a terrible traffic jam on the other side of the highway. Long lines of cars barely moving. As you keep driving, police car start appearing from afar shutting down the highway. Oh, it is an accident and there's a helicopter transporting the ones involved in the crash to the nearest hospital. They must be in critical condition for all of this to be happening. Now, wouldn't it be great if there is something in place that could warn you, given the weather and the road conditions about the possibility of you getting into a car accident and how severe it would be, so that you would drive more carefully or even change your travel if you are able to. Well, this is exactly what you will be working on in this course.

In [4]:
df = pd.read_csv("../Data-Collisions.csv")

/Users/strakh/Development/merapar/jupiterlab/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [6]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [7]:
df.shape

(194673, 38)

In [8]:
df.dtypes


SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [9]:
features = ['SEVERITYCODE', 'STATUS', 'ADDRTYPE', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'JUNCTIONTYPE', 'INATTENTIONIND', 'SPEEDING', 'HITPARKEDCAR', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'UNDERINFL']

In [10]:
X = df[features]

In [11]:
X.shape

(194673, 16)

In [12]:
X[['STATUS']].value_counts()

STATUS   
Matched      189786
Unmatched      4887
dtype: int64

In [13]:
X = X.loc[X['STATUS'] == 'Matched']
X[['STATUS']].value_counts()

STATUS 
Matched    189786
dtype: int64

In [14]:
X.shape

(189786, 16)

In [15]:
X = X.drop('STATUS', axis=1)
X.shape

(189786, 15)

In [16]:
X[['JUNCTIONTYPE']].value_counts()

JUNCTIONTYPE                                     
Mid-Block (not related to intersection)              87119
At Intersection (intersection related)               61334
Mid-Block (but intersection related)                 22388
Driveway Junction                                    10538
At Intersection (but not related to intersection)     2060
Ramp Junction                                          162
Unknown                                                  7
dtype: int64

In [17]:
X = X.loc[X['JUNCTIONTYPE'] != 'Unknown']

In [18]:
X.shape

(189779, 15)

In [19]:
for field in 'INATTENTIONIND', 'SPEEDING':
    print(X[[field]].value_counts())
    X[[field]] = X[[field]].fillna(0).replace('Y', 1)
    print(X[[field]].value_counts())

INATTENTIONIND
Y                 29805
dtype: int64
INATTENTIONIND
0                 159974
1                  29805
dtype: int64
SPEEDING
Y           9332
dtype: int64
SPEEDING
0           180447
1             9332
dtype: int64


In [20]:
X[['HITPARKEDCAR']] = X[['HITPARKEDCAR']].replace(('N','Y'), (0, 1))
X[['HITPARKEDCAR']].value_counts()

HITPARKEDCAR
0               182862
1                 6917
dtype: int64

In [21]:
for field in ('WEATHER', 'ROADCOND', 'LIGHTCOND'):
    print(X[[field]].value_counts())
    X = X.loc[~X[field].isin(('Unknown', 'Other'))]
    print(X[[field]].value_counts())

WEATHER                 
Clear                       111129
Raining                      33144
Overcast                     27712
Unknown                      15090
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
dtype: int64
WEATHER                 
Clear                       111129
Raining                      33144
Overcast                     27712
Snowing                        907
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
dtype: int64
ROADCOND      
Dry               123473
Wet                46978
Ice                 1116
Unknown              929
Snow/Slush           904
Standing Water       108
Other                108
Sand/Mud/Dirt         66
Oil      

In [22]:
X.shape

(170323, 15)

In [23]:
X[['LIGHTCOND']] = X[['LIGHTCOND']].replace(r"^Dark.+", "Dark", regex=True)
X[['LIGHTCOND']].value_counts()

LIGHTCOND
Daylight     112668
Dark          49348
Dusk           5660
Dawn           2412
dtype: int64

In [24]:
X[['UNDERINFL']].value_counts()

UNDERINFL
N            89822
0            71499
Y             5060
1             3942
dtype: int64

In [25]:
X[['UNDERINFL']] = X[['UNDERINFL']].replace(('N', 'Y'), (0,1)).astype('int')
print(X[['UNDERINFL']].value_counts())
X = X.loc[X['UNDERINFL'] == 0]
X.drop('UNDERINFL', axis=1, inplace=True)
X.shape

UNDERINFL
0            161321
1              9002
dtype: int64


(161321, 14)

In [26]:
X.dtypes

SEVERITYCODE       int64
ADDRTYPE          object
COLLISIONTYPE     object
PERSONCOUNT        int64
PEDCOUNT           int64
PEDCYLCOUNT        int64
VEHCOUNT           int64
JUNCTIONTYPE      object
INATTENTIONIND     int64
SPEEDING           int64
HITPARKEDCAR       int64
WEATHER           object
ROADCOND          object
LIGHTCOND         object
dtype: object

In [27]:
X.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,INATTENTIONIND,SPEEDING,HITPARKEDCAR,WEATHER,ROADCOND,LIGHTCOND
0,2,Intersection,Angles,2,0,0,2,At Intersection (intersection related),0,0,0,Overcast,Wet,Daylight
1,1,Block,Sideswipe,2,0,0,2,Mid-Block (not related to intersection),0,0,0,Raining,Wet,Dark
2,1,Block,Parked Car,4,0,0,3,Mid-Block (not related to intersection),0,0,0,Overcast,Dry,Daylight
3,1,Block,Other,3,0,0,3,Mid-Block (not related to intersection),0,0,0,Clear,Dry,Daylight
4,2,Intersection,Angles,2,0,0,2,At Intersection (intersection related),0,0,0,Raining,Wet,Daylight


In [28]:
for field in 'ADDRTYPE', 'COLLISIONTYPE', 'JUNCTIONTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND':
    X = pd.concat([X,pd.get_dummies(X[field], field)], axis=1)
    X.drop(field, axis=1, inplace=True)
    print(X.columns)

Index(['SEVERITYCODE', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT',
       'PEDCYLCOUNT', 'VEHCOUNT', 'JUNCTIONTYPE', 'INATTENTIONIND', 'SPEEDING',
       'HITPARKEDCAR', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'ADDRTYPE_Alley',
       'ADDRTYPE_Block', 'ADDRTYPE_Intersection'],
      dtype='object')
Index(['SEVERITYCODE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT',
       'JUNCTIONTYPE', 'INATTENTIONIND', 'SPEEDING', 'HITPARKEDCAR', 'WEATHER',
       'ROADCOND', 'LIGHTCOND', 'ADDRTYPE_Alley', 'ADDRTYPE_Block',
       'ADDRTYPE_Intersection', 'COLLISIONTYPE_Angles', 'COLLISIONTYPE_Cycles',
       'COLLISIONTYPE_Head On', 'COLLISIONTYPE_Left Turn',
       'COLLISIONTYPE_Other', 'COLLISIONTYPE_Parked Car',
       'COLLISIONTYPE_Pedestrian', 'COLLISIONTYPE_Rear Ended',
       'COLLISIONTYPE_Right Turn', 'COLLISIONTYPE_Sideswipe'],
      dtype='object')
Index(['SEVERITYCODE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT',
       'INATTENTIONIND', 'SPEEDING', 'HITPARKEDCAR', 'WE

In [29]:
X.shape

(161321, 47)

In [30]:
X.head()

,SEVERITYCODE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INATTENTIONIND,SPEEDING,HITPARKEDCAR,ADDRTYPE_Alley,ADDRTYPE_Block,...,ROADCOND_Ice,ROADCOND_Oil,ROADCOND_Sand/Mud/Dirt,ROADCOND_Snow/Slush,ROADCOND_Standing Water,ROADCOND_Wet,LIGHTCOND_Dark,LIGHTCOND_Dawn,LIGHTCOND_Daylight,LIGHTCOND_Dusk
0,2,2,0,0,2,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,1,2,0,0,2,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
2,1,4,0,0,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,3,0,0,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,2,2,0,0,2,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [31]:
X.describe()

,SEVERITYCODE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INATTENTIONIND,SPEEDING,HITPARKEDCAR,ADDRTYPE_Alley,ADDRTYPE_Block,...,ROADCOND_Ice,ROADCOND_Oil,ROADCOND_Sand/Mud/Dirt,ROADCOND_Snow/Slush,ROADCOND_Standing Water,ROADCOND_Wet,LIGHTCOND_Dark,LIGHTCOND_Dawn,LIGHTCOND_Daylight,LIGHTCOND_Dusk
count,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,...,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000,161321.000000
mean,1.323994,2.500177,0.040212,0.032246,1.971510,0.171304,0.048754,0.027579,0.003447,0.625666,...,0.006515,0.000372,0.000372,0.004909,0.000614,0.271254,0.263927,0.014499,0.686724,0.033461
std,0.467999,1.409160,0.205493,0.178017,0.554869,0.376776,0.215353,0.163762,0.058606,0.483952,...,0.080452,0.019282,0.019282,0.069896,0.024765,0.444608,0.440762,0.119536,0.463827,0.179838
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,2.000000,3.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000
max,2.000000,81.000000,5.000000,2.000000,12.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
X.dtypes

SEVERITYCODE                                                      int64
PERSONCOUNT                                                       int64
PEDCOUNT                                                          int64
PEDCYLCOUNT                                                       int64
VEHCOUNT                                                          int64
INATTENTIONIND                                                    int64
SPEEDING                                                          int64
HITPARKEDCAR                                                      int64
ADDRTYPE_Alley                                                    uint8
ADDRTYPE_Block                                                    uint8
ADDRTYPE_Intersection                                             uint8
COLLISIONTYPE_Angles                                              uint8
COLLISIONTYPE_Cycles                                              uint8
COLLISIONTYPE_Head On                                           

In [33]:
y = X[['SEVERITYCODE']]
X.drop('SEVERITYCODE', axis=1, inplace=True)
y.value_counts()


SEVERITYCODE
1               109054
2                52267
dtype: int64

In [34]:
y = y - 1
y.value_counts()

SEVERITYCODE
0               109054
1                52267
dtype: int64

In [35]:
c = X.corr()
c.head()

,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INATTENTIONIND,SPEEDING,HITPARKEDCAR,ADDRTYPE_Alley,ADDRTYPE_Block,ADDRTYPE_Intersection,...,ROADCOND_Ice,ROADCOND_Oil,ROADCOND_Sand/Mud/Dirt,ROADCOND_Snow/Slush,ROADCOND_Standing Water,ROADCOND_Wet,LIGHTCOND_Dark,LIGHTCOND_Dawn,LIGHTCOND_Daylight,LIGHTCOND_Dusk
PERSONCOUNT,1.000000,-0.033558,-0.046949,0.383268,0.067654,-0.009803,-0.047070,-0.023201,-0.046960,0.052238,...,-0.016168,-0.005021,-0.007759,-0.001393,-0.009329,0.002738,-0.011138,-0.024395,0.014975,0.005749
PEDCOUNT,-0.033558,1.000000,-0.020026,-0.328596,-0.010829,-0.034916,-0.030928,0.004448,-0.138794,0.140312,...,-0.008347,-0.003775,-0.000646,-0.003819,-0.002413,0.018616,0.024221,0.009828,-0.026455,0.002338
PEDCYLCOUNT,-0.046949,-0.020026,1.000000,-0.324878,-0.003061,-0.021767,-0.027103,-0.005899,-0.075961,0.078303,...,-0.009908,-0.003494,0.000118,-0.010731,-0.000271,-0.040888,-0.038629,0.009490,0.032557,0.004634
VEHCOUNT,0.383268,-0.328596,-0.324878,1.000000,0.053737,-0.046581,0.068134,-0.030530,0.093713,-0.089993,...,-0.018754,-0.009439,-0.014653,-0.004225,-0.016321,-0.013878,-0.039287,-0.016857,0.041986,-0.000759
INATTENTIONIND,0.067654,-0.010829,-0.003061,0.053737,1.000000,-0.055335,0.019879,-0.004280,0.102930,-0.101387,...,-0.022094,-0.007063,-0.007063,-0.017577,-0.001302,-0.029163,-0.055716,-0.001195,0.057228,-0.010310


In [36]:
c = c.abs().unstack().sort_values(ascending=False).drop_duplicates()
c = c.loc[(c > 0.9) & (c != 1.0)]
c

PEDCYLCOUNT                                          COLLISIONTYPE_Cycles                                   0.987620
ADDRTYPE_Block                                       ADDRTYPE_Intersection                                  0.983682
JUNCTIONTYPE_At Intersection (intersection related)  ADDRTYPE_Intersection                                  0.976191
ROADCOND_Wet                                         ROADCOND_Dry                                           0.967011
ADDRTYPE_Block                                       JUNCTIONTYPE_At Intersection (intersection related)    0.960604
COLLISIONTYPE_Pedestrian                             PEDCOUNT                                               0.947454
dtype: float64

In [37]:
to_drop = {item[0] for item in c.index}
print(to_drop)
for field in to_drop:
    X.drop(field, axis=1, inplace=True)


{'COLLISIONTYPE_Pedestrian', 'PEDCYLCOUNT', 'ROADCOND_Wet', 'ADDRTYPE_Block', 'JUNCTIONTYPE_At Intersection (intersection related)'}


In [38]:
print(X.shape)
y = y.values.ravel()
print(y.shape)

(161321, 41)
(161321,)


In [39]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))

In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

print("number of test samples :", X_test.shape[0])
print("number of training samples:",X_train.shape[0])

number of test samples : 3227
number of training samples: 12905


In [41]:
results = {}
class Result:
    def __init__(self, confusion, accuracy, f1, logloss=None):
        self.confusion = confusion
        self.accuracy = accuracy
        self.f1 = f1
        self.logloss = logloss

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
for depth in range(1, 20):
    dtree = DecisionTreeClassifier(criterion="entropy", max_depth = depth).fit(X_train, y_train)
    print(f"DecisionTrees's Accuracy (depth={depth}): {metrics.accuracy_score(y_test, dtree.predict(X_test))}")

DecisionTrees's Accuracy (depth=1): 0.6629491945477075
DecisionTrees's Accuracy (depth=2): 0.6957868649318464
DecisionTrees's Accuracy (depth=3): 0.7280049566294919
DecisionTrees's Accuracy (depth=4): 0.7280049566294919
DecisionTrees's Accuracy (depth=5): 0.7273853779429987
DecisionTrees's Accuracy (depth=6): 0.7317224287484511
DecisionTrees's Accuracy (depth=7): 0.7242874845105328
DecisionTrees's Accuracy (depth=8): 0.7267657992565055
DecisionTrees's Accuracy (depth=9): 0.7218091697645601
DecisionTrees's Accuracy (depth=10): 0.7236679058240396
DecisionTrees's Accuracy (depth=11): 0.7156133828996283
DecisionTrees's Accuracy (depth=12): 0.7094175960346965
DecisionTrees's Accuracy (depth=13): 0.7137546468401487
DecisionTrees's Accuracy (depth=14): 0.7149938042131351
DecisionTrees's Accuracy (depth=15): 0.7094175960346965
DecisionTrees's Accuracy (depth=16): 0.7069392812887236
DecisionTrees's Accuracy (depth=17): 0.701363073110285
DecisionTrees's Accuracy (depth=18): 0.7032218091697645
De

In [66]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
for depth in range(1, 20):
    rfc = RandomForestClassifier(max_depth=depth, random_state=0).fit(X_train, y_train)
    print(f"RandomForest's Accuracy (depth={depth}): {metrics.accuracy_score(y_test, rfc.predict(X_test))}")

RandomForest's Accuracy (depth=1): 0.6755500464828014
RandomForest's Accuracy (depth=2): 0.7238921599008367
RandomForest's Accuracy (depth=3): 0.7356678029129222
RandomForest's Accuracy (depth=4): 0.7356678029129222
RandomForest's Accuracy (depth=5): 0.7362875735977689
RandomForest's Accuracy (depth=6): 0.7365974589401921
RandomForest's Accuracy (depth=7): 0.7390765416795786
RandomForest's Accuracy (depth=8): 0.7400061977068485
RandomForest's Accuracy (depth=9): 0.7424852804462349
RandomForest's Accuracy (depth=10): 0.7387666563371552
RandomForest's Accuracy (depth=11): 0.7403160830492718
RandomForest's Accuracy (depth=12): 0.7378370003098853
RandomForest's Accuracy (depth=13): 0.7362875735977689
RandomForest's Accuracy (depth=14): 0.7316392934614193
RandomForest's Accuracy (depth=15): 0.7313294081189959
RandomForest's Accuracy (depth=16): 0.7279206693523397
RandomForest's Accuracy (depth=17): 0.7282305546947629
RandomForest's Accuracy (depth=18): 0.7217229625038736
RandomForest's Accu

In [44]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
for n in range(1, 30):
    kmeans = KNeighborsClassifier(n_neighbors = n).fit(X_train, y_train)
    print(f"K-Means's Accuracy (n={n}): {metrics.accuracy_score(y_test, kmeans.predict(X_test))}")

K-Means's Accuracy (n=1): 0.6493184634448576
K-Means's Accuracy (n=2): 0.6995043370508055
K-Means's Accuracy (n=3): 0.6951672862453532
K-Means's Accuracy (n=4): 0.7069392812887236
K-Means's Accuracy (n=5): 0.69454770755886
K-Means's Accuracy (n=6): 0.7162329615861215
K-Means's Accuracy (n=7): 0.7106567534076828
K-Means's Accuracy (n=8): 0.7174721189591078
K-Means's Accuracy (n=9): 0.7162329615861215
K-Means's Accuracy (n=10): 0.7224287484510533
K-Means's Accuracy (n=11): 0.7255266418835192
K-Means's Accuracy (n=12): 0.7292441140024783
K-Means's Accuracy (n=13): 0.7280049566294919
K-Means's Accuracy (n=14): 0.7267657992565055
K-Means's Accuracy (n=15): 0.7267657992565055
K-Means's Accuracy (n=16): 0.7342007434944238
K-Means's Accuracy (n=17): 0.7261462205700124
K-Means's Accuracy (n=18): 0.7354399008674102
K-Means's Accuracy (n=19): 0.7342007434944238
K-Means's Accuracy (n=20): 0.7354399008674102
K-Means's Accuracy (n=21): 0.7360594795539034
K-Means's Accuracy (n=22): 0.7354399008674102

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
solvers = ("newton-cg", "lbfgs", "liblinear", "sag", "saga")
cs = (1, 0.5, 0.1, 0.05, 0.01)
for i, solver in enumerate(solvers):
    for j, c in enumerate(cs):
        lr = LogisticRegression(C=c, solver=solver).fit(X_train, y_train)
#         print(f"LogReg's Accuracy score (solver={solver}, C={c}): {metrics.accuracy_score(y_test, kmeans.predict(X_test))}")
        print(f"LogReg's F1 score (solver={solver}, C={c}): {metrics.f1_score(y_test, kmeans.predict(X_test), average='weighted')}")

LogReg's F1 score (solver=newton-cg, C=1): 0.6978180900597674
LogReg's F1 score (solver=newton-cg, C=0.5): 0.6978180900597674
LogReg's F1 score (solver=newton-cg, C=0.1): 0.6978180900597674
LogReg's F1 score (solver=newton-cg, C=0.05): 0.6978180900597674
LogReg's F1 score (solver=newton-cg, C=0.01): 0.6978180900597674
LogReg's F1 score (solver=lbfgs, C=1): 0.6978180900597674
LogReg's F1 score (solver=lbfgs, C=0.5): 0.6978180900597674
LogReg's F1 score (solver=lbfgs, C=0.1): 0.6978180900597674
LogReg's F1 score (solver=lbfgs, C=0.05): 0.6978180900597674
LogReg's F1 score (solver=lbfgs, C=0.01): 0.6978180900597674
LogReg's F1 score (solver=liblinear, C=1): 0.6978180900597674
LogReg's F1 score (solver=liblinear, C=0.5): 0.6978180900597674
LogReg's F1 score (solver=liblinear, C=0.1): 0.6978180900597674
LogReg's F1 score (solver=liblinear, C=0.05): 0.6978180900597674
LogReg's F1 score (solver=liblinear, C=0.01): 0.6978180900597674


/Users/strakh/Development/merapar/jupiterlab/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogReg's F1 score (solver=sag, C=1): 0.6978180900597674


/Users/strakh/Development/merapar/jupiterlab/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogReg's F1 score (solver=sag, C=0.5): 0.6978180900597674


...

LogReg's F1 score (solver=saga, C=0.5): 0.6978180900597674


/Users/strakh/Development/merapar/jupiterlab/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogReg's F1 score (solver=saga, C=0.1): 0.6978180900597674


/Users/strakh/Development/merapar/jupiterlab/env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogReg's F1 score (solver=saga, C=0.05): 0.6978180900597674
LogReg's F1 score (solver=saga, C=0.01): 0.6978180900597674


In [46]:
from sklearn import svm
from sklearn import metrics
kernels = ("linear", "poly", "rbf", "sigmoid")
for i, kernel in enumerate(kernels):
    svc = svm.SVC(kernel=kernel, gamma="auto").fit(X_train, y_train)
    print(f"SVM's F1 score (kernel={kernel}): {metrics.f1_score(y_test, svc.predict(X_test), average='weighted')}")


SVM's F1 score (kernel=linear): 0.6638353549033312
SVM's F1 score (kernel=poly): 0.6664669730806586
SVM's F1 score (kernel=rbf): 0.6757734462987592
SVM's F1 score (kernel=sigmoid): 0.6703043368731163


In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print("number of test samples :", X_test.shape[0])
print("number of training samples:",X_train.shape[0])

number of test samples : 32265
number of training samples: 129056


In [48]:
depth=6
dtree = DecisionTreeClassifier(criterion="entropy", max_depth = depth).fit(X_train, y_train)
y_predict = dtree.predict(X_test)
results[dtree] = Result(
    metrics.confusion_matrix(y_test, y_predict),
    metrics.accuracy_score(y_test, y_predict),
    metrics.f1_score(y_test, y_predict, average='weighted')
    )
print(metrics.classification_report(y_test, y_predict))
print(metrics.classification_report(y_train, dtree.predict(X_train)))

              precision    recall  f1-score   support

           0       0.74      0.95      0.83     21854
           1       0.74      0.28      0.41     10411

    accuracy                           0.74     32265
   macro avg       0.74      0.62      0.62     32265
weighted avg       0.74      0.74      0.69     32265

              precision    recall  f1-score   support

           0       0.74      0.95      0.83     87200
           1       0.74      0.29      0.42     41856

    accuracy                           0.74    129056
   macro avg       0.74      0.62      0.62    129056
weighted avg       0.74      0.74      0.70    129056



In [49]:
n = 21
kmeans = KNeighborsClassifier(n_neighbors = n).fit(X_train, y_train)
y_predict = kmeans.predict(X_test)
results[kmeans] = Result(
    metrics.confusion_matrix(y_test, y_predict),
    metrics.accuracy_score(y_test, y_predict),
    metrics.f1_score(y_test, y_predict, average='weighted')
    )
print(metrics.classification_report(y_test, y_predict))
print(metrics.classification_report(y_train, kmeans.predict(X_train)))

              precision    recall  f1-score   support

           0       0.75      0.90      0.82     21854
           1       0.64      0.36      0.46     10411

    accuracy                           0.73     32265
   macro avg       0.70      0.63      0.64     32265
weighted avg       0.71      0.73      0.70     32265

              precision    recall  f1-score   support

           0       0.75      0.91      0.82     87200
           1       0.67      0.38      0.48     41856

    accuracy                           0.74    129056
   macro avg       0.71      0.64      0.65    129056
weighted avg       0.72      0.74      0.71    129056



In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
lr = LogisticRegression().fit(X_train, y_train)
y_predict = lr.predict(X_test)
results[lr] = Result(
    metrics.confusion_matrix(y_test, y_predict),
    metrics.accuracy_score(y_test, y_predict),
    metrics.f1_score(y_test, y_predict, average='weighted'),
    metrics.log_loss(y_test, lr.predict_proba(X_test))
    )
print(metrics.classification_report(y_test, y_predict))
print(metrics.classification_report(y_train, lr.predict(X_train)))

              precision    recall  f1-score   support

           0       0.74      0.96      0.83     21854
           1       0.76      0.28      0.41     10411

    accuracy                           0.74     32265
   macro avg       0.75      0.62      0.62     32265
weighted avg       0.74      0.74      0.70     32265

              precision    recall  f1-score   support

           0       0.74      0.95      0.83     87200
           1       0.75      0.29      0.41     41856

    accuracy                           0.74    129056
   macro avg       0.74      0.62      0.62    129056
weighted avg       0.74      0.74      0.70    129056



In [51]:
kernel="rbf"
svc = svm.SVC(kernel=kernel, gamma="auto").fit(X_train, y_train)
y_predict = svc.predict(X_test)
results[svc] = Result(
    metrics.confusion_matrix(y_test, y_predict),
    metrics.accuracy_score(y_test, y_predict),
    metrics.f1_score(y_test, y_predict, average='weighted')
    )
print(metrics.classification_report(y_test, y_predict))
print(metrics.classification_report(y_train, svc.predict(X_train)))

              precision    recall  f1-score   support

           0       0.74      0.96      0.83     21854
           1       0.75      0.28      0.41     10411

    accuracy                           0.74     32265
   macro avg       0.75      0.62      0.62     32265
weighted avg       0.74      0.74      0.70     32265

              precision    recall  f1-score   support

           0       0.74      0.96      0.83     87200
           1       0.76      0.29      0.42     41856

    accuracy                           0.74    129056
   macro avg       0.75      0.62      0.63    129056
weighted avg       0.75      0.74      0.70    129056



In [52]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
gbc = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
y_predict = gbc.predict(X_test)
results[gbc] = Result(
    metrics.confusion_matrix(y_test, y_predict),
    metrics.accuracy_score(y_test, y_predict),
    metrics.f1_score(y_test, y_predict, average='weighted')
    )
print(metrics.classification_report(y_test, y_predict))
print(metrics.classification_report(y_train, gbc.predict(X_train)))

              precision    recall  f1-score   support

           0       0.74      0.95      0.83     21854
           1       0.74      0.30      0.43     10411

    accuracy                           0.74     32265
   macro avg       0.74      0.63      0.63     32265
weighted avg       0.74      0.74      0.70     32265

              precision    recall  f1-score   support

           0       0.74      0.95      0.83     87200
           1       0.74      0.31      0.43     41856

    accuracy                           0.74    129056
   macro avg       0.74      0.63      0.63    129056
weighted avg       0.74      0.74      0.70    129056



In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

depth = 9
rfc = RandomForestClassifier(max_depth=depth, random_state=0).fit(X_train, y_train)
y_predict = rfc.predict(X_test)
results[rfc] = Result(
    metrics.confusion_matrix(y_test, y_predict),
    metrics.accuracy_score(y_test, y_predict),
    metrics.f1_score(y_test, y_predict, average='weighted')
    )
print(metrics.classification_report(y_test, y_predict))
print(metrics.classification_report(y_train, rfc.predict(X_train)))

              precision    recall  f1-score   support

           0       0.73      0.98      0.83     21854
           1       0.82      0.24      0.37     10411

    accuracy                           0.74     32265
   macro avg       0.78      0.61      0.60     32265
weighted avg       0.76      0.74      0.68     32265

              precision    recall  f1-score   support

           0       0.73      0.98      0.83     87200
           1       0.83      0.24      0.37     41856

    accuracy                           0.74    129056
   macro avg       0.78      0.61      0.60    129056
weighted avg       0.76      0.74      0.68    129056



In [54]:
# from sklearn.naive_bayes import ComplementNB
# from sklearn import metrics
# nb = ComplementNB().fit(X_train, y_train)
# y_predict = nb.predict(X_test)
# print(metrics.confusion_matrix(y_test, y_predict))
# print(f"RandomForest's Accuracy score: {metrics.accuracy_score(y_test, y_predict)}")
# print(f"RandomForest's F1 score: {metrics.f1_score(y_test, y_predict, average='weighted')}")
# print(metrics.classification_report(y_test, y_predict))
# print(metrics.classification_report(y_train, nb.predict(X_train)))

In [73]:
res = {}
for model, result in results.items():
    name = model.__class__.__name__
    res[name] = result.__dict__
pd.DataFrame.from_dict(res).transpose()


,confusion,accuracy,f1,logloss
DecisionTreeClassifier,"[[20791, 1063], [7450, 2961]]",0.736154,0.694602,None
KNeighborsClassifier,"[[19762, 2092], [6638, 3773]]",0.729428,0.704387,None
LogisticRegression,"[[20922, 932], [7454, 2957]]",0.74009,0.697693,0.510729
SVC,"[[20903, 951], [7484, 2927]]",0.738571,0.695805,None
GradientBoostingClassifier,"[[20733, 1121], [7260, 3151]]",0.740245,0.701938,None
RandomForestClassifier,"[[21318, 536], [7935, 2476]]",0.737455,0.684101,None


In [74]:
for model, result in res.items():
    print(model)
    print(result["confusion"])

DecisionTreeClassifier
[[20791  1063]
 [ 7450  2961]]
KNeighborsClassifier
[[19762  2092]
 [ 6638  3773]]
LogisticRegression
[[20922   932]
 [ 7454  2957]]
SVC
[[20903   951]
 [ 7484  2927]]
GradientBoostingClassifier
[[20733  1121]
 [ 7260  3151]]
RandomForestClassifier
[[21318   536]
 [ 7935  2476]]
